Fonte: [documentação pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html)

In [2]:
%config Completer.use_jedi = False

import pandas as pd

# Leitura de dados com Pandas

Vamos aprender aqui o ```read_csv```, ```read_excel``` e o ```read_clipboard```. Claro que existem outros, mas escolhemos esses por ser os mais utilizados (o uso entre eles é bem parecido).



---
## Lendo arquivos CSV

Para ler um arquivo CSV é bem tranquilo:

In [ ]:
df = pd.read_csv('dados/titanic.csv')
df.head()

Também temos alguns argumentos que podem nos ajudar em alguns casos:
* sep: separador do arquivo CSV;
* usecols: as colunas que você quer ler do CSV;
* nrows: a quantidade de linhas que você quer ler;
* skiprows: as linhas que você não quer ler ao carregar o arquivo;
* dentre outros...

In [ ]:
usecols = ['PassengerId', 'Survived', 'Pclass']
df = pd.read_csv('./dados/titanic.csv', sep=',', usecols=usecols, nrows=2, skiprows=[1, 2])
df.head()

Também podemos ler CSV's em chunks. Onde você carrega uma quantidade ```n``` de linhas por vez e processa, com o objetivo de economizar recursos ou viabilizar a execução pela falta de recursos. 

In [ ]:
# onde o chunksize é a quantidade de linhas lidas por vez
df_ = pd.read_csv('./dados/titanic.csv', chunksize=200)
for df in df_:
    # processamento: aqui poderia ser a chamada de uma função ou qualquer outro processamento
    value_counts = df['Survived'].value_counts(normalize=True)
    print(f'valuecounts: \n{value_counts} \n')

----
## Lendo arquivos excel

In [ ]:
df = pd.read_excel('./dados/titanic.xlsx')
df.head()

Podemos ler um arquivo excel que tem "abas" da seguinte maneira:

In [ ]:
df_dict = pd.read_excel('./dados/titanic.xlsx', sheet_name=None)

Onde o resultado obtido é um dicionário, onde a chave é o nome da aba e os conteúdos são os próprios DataFrames referentes às abas:

In [ ]:
isinstance(df_dict, dict)

In [ ]:
df_dict.keys()

Primeira aba:

In [ ]:
df_dict['exemplo_completo'].head()

Segunda aba:

In [ ]:
df_dict['exemplo_cortado'].head()

---
## Lendo arquivos da área de transferência (ctrl+C)

Podemos também, ler um arquivo (uma tabela) da área de transferência:

In [ ]:
df = pd.read_clipboard()
df.head()

---

### Salvando um arquivo

Podemos também, salvar um arquivo facilmente fazendo:

In [ ]:
# é importante o index ser None para não salvá-lo no arquivo
df.to_excel('./dados/area_transferencia_exemplo.xlsx', index=None)
# ou
df.to_csv('./dados/area_transferencia_exemplo.csv', index=None, sep=',')

# Manipulação de DataFrames

Vamos criar alguns DataFrames exemplo para treinarmos:

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)

Vamos printar os DataFrames pra ver como que eles são:

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3.head()

## Concat (pd.concat)

### Cruzamento vertical

Podemos perceber que eles tem as mesmas colunas, então faria sentido "juntar" esses DataFrames verticalmente (ou seja, __em relação às linhas__), caso preciso.  
Vamos fazer isso?

Para executar essa ação, vamos criar uma lista chamada ```frames```, onde o conteúdo dela vai ser nossos DataFrames.  
Em seguida, precisamos chamar a função ```pd.concat()```, para juntar esses DataFrames __verticalmente__.

In [ ]:
frames = [df1, df2, df3]
df_resultado = pd.concat(frames)
df_resultado

Eventualmente, a gente precisa saber a fonte de dados de cada uma das linhas. É possível relacionar isso com uma "chave", utilizando o argumento ```keys```:

In [ ]:
keys = ['base'+str(idx) for idx in range(len(frames))]
df_resultado = pd.concat(frames, keys=keys)
df_resultado

### Cruzamento horizontal

Primeiramente, precisamos criar um DataFrame novo para o exemplo:

In [ ]:
df4 = pd.DataFrame(
    {
        "B": ["B2", "B3", "B6", "B7"],
        "D": ["D2", "D3", "D6", "D7"],
        "F": ["F2", "F3", "F6", "F7"],
    },
    index=[2, 3, 6, 7],
)
df4.head()

In [ ]:
df1.head()

Dados os DataFrames, podemos cruzar os dois __horizontalmente__ (ou seja, em relação às colunas), trocando o valor do ```axis=0``` (padrão) para ```axis=1```:

In [ ]:
df_resultado = pd.concat([df1, df4], axis=1)
df_resultado

É legal observar, que ao fazer o cruzamento, o DataFrame resultado gerou algumas colunas com o famoso ```NaN```, por que?  
  
O motivo é o fator de cruzarmento ser o __outer__, assim, a função preenche lugares que não teve _match_ com o ```NaN```, adicionando as linhas necessárias (a quantidade de linhas vai ser ```qte_linhas_df1``` + ```qte_linhas_df5```).

Mas é possível também cruzar apenas os _matches_! Mas, como?  
Fazemos  a mesma coisa, mas mudamos o argumento ```join='outer'``` para ```join='inner'```:

In [ ]:
df_resultado = pd.concat([df1, df4], axis=1, join='inner')
df_resultado

Podemos também, ignorar o fato de existir um index, e apenas empilhar os dados:

In [ ]:
df_resultado = pd.concat([df1, df4], ignore_index=True, sort=False)
df_resultado

### Extras

Criando um DataFrame a partir de Series com o concat:

In [ ]:
# criando as Series:
s1 = pd.Series(['a1', 'a2', 'a3', 'a4'])

s2 = pd.Series(['b1', 'b2', 'b3', 'b4'])

s3 = pd.Series(['c1', 'c2', 'c3', 'c4'])

df_resultado = pd.concat([s1, s2, s3], axis=1)
df_resultado

Podemos passar o nome das colunas no argumento ```keys```:

In [ ]:
keys=["a", "b", "c"]

df_resultado = pd.concat([s1, s2, s3], axis=1, keys=keys)
df_resultado

## Concat (append)

Existe também, uma função do próprio DataFrame, que tem o nome de ```pd.DataFrame().append()```. O objetivo da função é o mesmo que o concat, e ela é muito parecida com o ```list.append()``` que vimos em listas:

In [ ]:
df_resultado = df1.append(df2)
df_resultado

Conseguimos fazer a mesma coisa, mas com uma lista de DataFrames:

In [ ]:
df_resultado = df1.append([df2, df3])
df_resultado

Verticalmente vimos que funciona muito bem! E horizontalmente? Vamos tentar? 

In [ ]:
df_resultado = df1.append(df4, sort=False)
df_resultado

Não foi o resultado esperado, né? Ele lidou muito bem com o cruzamento das colunas, mas simplesmente empilhou as linhas... =/

### Extra

É possível adicionar apenas uma Series ao DataFrame:

In [ ]:
# é importante ter o index setado para não ocorrer erros
sX = pd.Series(["X0", "X1", "X2", "X3"], index=["A", "B", "C", "D"])

df_resultado = df1.append(sX, ignore_index=True)
df_resultado

## Merge

Veremos agora, um outros método de cruzamento, muito performático do pandas.  
  
Para isso, vamos criar dois DataFrames novos:

In [3]:
df_left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)


df_right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [4]:
df_left.head()

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [5]:
df_right.head()

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


Conseguimos cruzar horizontalmente (muito parecido com cruzamentos que vamos ver em banco de dados):

In [6]:
df_resultado

NameError: name 'df_resultado' is not defined

Onde o parâmetro ```on``` é a chave que ele usa para o cruzamento. Então, para cruzar, ele vai verificar a seguinte condição:  
```df_left.key == df_right.key```

E se quisermos utilizar duas chaves?  
Podemos evoluir a condição anterior para:  
```(df_left.key1 == df_right.key1) E (df_left.key2 == df_right.key2)```,  
E assim por diante... Podemos colocar quantas colunas quisermos! Limitados apenas pela quantidade de colunas dos DF's.

Exemplo:

In [ ]:
df_left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)


df_right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [ ]:
df_left

In [ ]:
df_right

In [7]:
df_resultado = pd.merge(df_left, df_right, on=["key1", "key2"])
df_resultado

KeyError: 'key1'

O argumento ```how```, nos diz qual método vai ser utilizado para fazer o cruzamento:  
```P.S.: Tomei a liberdade de traduzir essa tabela da própria documentação do pandas.```

<table class="colwidths-given table">
<colgroup>
<col style="width: 20%">
<col style="width: 20%">
<col style="width: 60%">
</colgroup>
<thead>
<tr class="row-odd"><th class="head"><p>Método</p></th>
<th class="head"><p>SQL Join</p></th>
<th class="head"><p>Descrição</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">left</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">LEFT</span> <span class="pre">OUTER</span> <span class="pre">JOIN</span></code></p></td>
<td><p>Usa apenas as chaves do DataFrame da esquerda</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">right</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">RIGHT</span> <span class="pre">OUTER</span> <span class="pre">JOIN</span></code></p></td>
<td><p>Usa apenas as chaves do DataFrame da direita</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">outer</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">FULL</span> <span class="pre">OUTER</span> <span class="pre">JOIN</span></code></p></td>
<td><p>Usa a união das chaves dos DataFrames</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">inner</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">INNER</span> <span class="pre">JOIN</span></code></p></td>
<td><p>Usa a intersecção das chaves dos DataFrames</p></td>
</tr>
</tbody>
</table>

Exemplo:

In [ ]:
df_resultado = pd.merge(df_left, df_right, how='left', on=["key1", "key2"])
df_resultado

## Apply

In [60]:
df_titanic = pd.read_csv('./dados/titanic.csv')
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Vamos supor que que queremos separar as pessoas pela idade. Ou seja:
* __True__ se tiver mais de 12 anos 
* __False__ se tiver menos de 12 anos

Para fazer isso, vamos utilizar uma funcionalidade do python feita para funções enxutas, que se chama ```lambda```:

In [61]:
# lambda_exemplo = lambda idade: True if(idade>=12) else False
lambda_exemplo = lambda idade: idade>=12

In [62]:
print(lambda_exemplo(12))

True


Mas como aplicamos isso em todo o nosso DataFrame?  
Existe uma função, para as Series, chamada ```apply```:

In [63]:
df_titanic['Age'].apply(lambda idade: idade>=12)

0       True
1       True
2       True
3       True
4       True
       ...  
886     True
887     True
888    False
889     True
890     True
Name: Age, Length: 891, dtype: bool

O que a função ```apply``` faz?  
Ela aplica a função desejada (com um único argumento) para todas as linhas da Series.

In [ ]:
df_titanic['Age_processada_lambda'] = df_titanic['Age'].apply(lambda idade: idade>=12)
df_titanic.head()

Ao invés do ```lambda```, podemos utilizar também uma função que fizemos, ou uma já pronta:

In [ ]:
# def processa_idade(idade):
#     if(idade>=12):
#         return True
#     else:
#         return False
def processa_idade(idade):
    return idade>=12

In [ ]:
df_titanic['Age'].apply(processa_idade)

In [ ]:
df_titanic['Age_processada_funcao'] = df_titanic['Age'].apply(lambda idade: idade>=12)
df_titanic.head()

Vamos supor que queremos saber a quantidade de caracteres do nome... Podemos utilizar a função nativa do python ```len```, utilizando assim, uma função pronta do python:

In [ ]:
df_titanic['Name'].apply(len)

In [ ]:
df_titanic['tamanho_texte_nome'] = df_titanic['Name'].apply(len)
df_titanic.head()

Podemos utilizar o apply no DataFrame em si também:

In [ ]:
def idade_sobreviveu(linha):
    if(linha['Age_processada_funcao']==True and linha['Survived']==1):
        return True
    else:
        return False
def idade_sobreviveu(linha):
    return linha['Age_processada_funcao']==True and linha['Survived']==1

In [ ]:
df_titanic.apply(idade_sobreviveu, axis=1)

Lembrando sempre de especificar o ```axis=1``` para a aplicação ser nas linhas e não nas colunas:

In [ ]:
df_titanic['idade_sobreviveu'] = df_titanic.apply(idade_sobreviveu, axis=1)
df_titanic.head(3)

## Groupby

Vamos supor que queremos saber quantas pessoas sobreviveram dada a idade processada:

In [64]:
df_titanic.groupby(by=['Age_processada_funcao'])['Survived'].sum()

KeyError: 'Age_processada_funcao'

Também é possível transformar o resultado em um DataFrame, colocando ao invés de uma lista, uma lista dentro de lista mais a direita:

In [ ]:
#                                                    AQUI
df_titanic.groupby(by=['Age_processada_funcao'])[['Survived']].size()

E também, podemos fazer o groupby por várias features, ao invés de uma só:

In [ ]:
df_titanic.groupby(by=['Age_processada_funcao','Sex','Survived']).size()